`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `02`. Парсинг даных: HTML, DOM, __XPath__


## Задание 1. XPath. Лента.РУ

Написать приложение или функцию, которые собирают основные новости с сайта на выбор:
 * lenta.ru, 
 * yandex-новости. 
 
Для парсинга использовать `XPath`. Структура данных в виде словаря должна содержать:
- *название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

Минимум один сайт, максимум — все два


In [1]:
import pandas as pd
from pprint import pprint 
import datetime

import requests
from lxml import html


### Забираем все данные с сайта `Lenta.ru`

 Обращаемся к сайту как Хром методом `get` модуля `requests`. Кроме того:
 * необходимо запомнить дату текущего запроса
 * время этого текущего запроса
 * для продолжения работы необходимо уточнить код ответа сервера `status_code`
 * а так же необходимо уточнить, вернул ли нам сервер данные в формате `html`, информцию об этом сервер возвращает в заголовках `headers` в параметре `'Content-Type'`
 

In [2]:
%%time
url = 'https://lenta.ru/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

response = requests.get(url=url, headers=headers)

news_date = str(datetime.date.today())
news_time = str(datetime.datetime.now().strftime("%H:%M"))

response.status_code, response.headers['Content-Type'], response.url, 

Wall time: 80.1 ms


(200, 'text/html; charset=utf-8', 'https://lenta.ru/')

#### Перобразуем данные к формату `DOM`

Преобразование тела документа как строки в ответе `response.text` в _дерево_ `DOM` при помощи метода `fromstring` модуля `html` 

In [3]:
dom = html.fromstring(response.text) 

### Парсим
Выберем из дерева `DOM` только блоки с новостями. Каждый такой блок новостей — это объект. Все объекты объеденены в список `list`. При этом самым верхним уровнем в структуре для такого объекта будет не начало сайта или дерева `DOM`, а начало этого информационного блока — тэг `a`.

#### Функция
Фукнция для выборки парснутых данных в новостные элементы 

In [4]:
def eject(elements):
    ''' забираем первый элемент из списка'''
    if isinstance(elements, list):
        if len(elements) == 1 or len(elements) > 1:
            element = str(elements[0])
        else:
            element = None
    return element

#### Блоки МИНИ 
Класс `card-mini__title` с минимальными новостями из нашего `DOM` в виде списка объектов:
* Название новости
* Ссылку
* Время на текущий день. 
    - Новостной блок не содержит дату, а только время. Но мы располагаем датой запроса этих новостей, которую мы сохранили сразу после запроса в переменной `news_date`. 
    - Если время отсутствует в блоке с новостью, то воспользуемся временем запроса к сайту, сохранённого в переменной `news_time`.

__Блоки МИНИ__: все объекты с минимальными новостями

In [5]:
news_mini = dom.xpath('.//a[contains(@class, "card-mini")]')
print(f"Блоки МИНИ: количество: {len(news_mini)}")

Блоки МИНИ: количество: 89


__Блоки МИНИ__: Парсим данные для кадой новости и сохраняем их как строку `DataFrame`.

In [6]:
news_mini_table = pd.DataFrame(columns=['Name', 'Link', 'Date', 'Source', 'N_Type']) # инициализирум выходную таблицу
news_type = "mini"

for i in news_mini:
    name = eject(i.xpath('.//div[contains(@class, "card-mini__text")]/span[contains(@class, "card-mini__title")]/text()'))
    link = url.strip('/') + eject(i.xpath('.//@href'))
    time = eject(i.xpath('.//div[contains(@class, "card-mini__text")]/div[contains(@class, "card-mini__info")]/time[contains(@class, "card-mini__date")]/text()'))
    
    date = news_date + ';' + (time if time else news_time) 

    news_mini_table = news_mini_table.append({'Date': date, 'Name': name, 'Link': link, 'N_Type': news_type}, ignore_index=True)

print(f"НОВОСТИ МИНИ: {len(news_mini_table)} позиция распарсена")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# news_mini_table

НОВОСТИ МИНИ: 89 позиция распарсена


#### Блоки БИГ
Новостные блоки БИГ: класс объектов `card-mini__big` с расширенными новостями из нашего `DOM` в виде 2-ого списка `list`.

In [7]:
news_big = dom.xpath('.//a[contains(@class, "card-big")]')
print(f"Блоки БИГ, количество: {len(news_big)} объекта получено")

Блоки БИГ, количество: 53 объекта получено


In [8]:
news_big_table = pd.DataFrame(columns=['Name', 'Link', 'Date', 'Source', 'N_Type'], index=[]) # инициализирум выходную таблицу
news_type = "BIG"

for i in news_big:
    title = eject(i.xpath('.//div[contains(@class, "card-big__titles")]/h3[contains(@class, "card-big__title")]/text()'))
    notes = eject(i.xpath('.//div[contains(@class, "card-big__titles")]/span[@class]/text()'))
    time =  eject(i.xpath('.//div[contains(@class, "card-big__info")]/time[contains(@class, "card-big__date")]/text()'))
    link =  eject(i.xpath('.//@href'))
    src =   eject(i.xpath('.//div[contains(@class, "card-big__info")]/div/svg/use/@href'))   # xlink:href="#ui-label_motor"

    name  = title + (notes if notes else "")
    date = time
    link = url.strip("/") + link
    
    news_big_table = news_big_table.append({'Name': name, 'Link': link, 'Date': date, 'N_Type': news_type}, ignore_index=True)

    
print(f"НОВОСТИ БИГ: {len(news_big_table)} позиция распарсена")
# news_big_table

НОВОСТИ БИГ: 53 позиция распарсена


### ВСЕ НОВОСТИ: МИНИ + БИГ

In [9]:
news_table = news_mini_table.append(news_big_table, ignore_index=True)

In [10]:
print(f"ВСЕГО новостей: {len(news_table)}")
news_table

ВСЕГО новостей: 142


,Name,Link,Date,Source,N_Type
0,Отдыхающие с риском для жизни спасли самую быс...,https://lenta.ru/news/2022/09/29/sharkrescue/,2022-09-29;18:36,NaN,mini
1,Итальянский чемпион ОИ назвал позорным отстран...,https://lenta.ru/news/2022/09/29/milanoli/,2022-09-29;18:36,NaN,mini
2,Названы главные критерии выбора хорошего пласт...,https://lenta.ru/news/2022/09/29/vyborhirurga/,2022-09-29;18:35,NaN,mini
3,МВД отвергло обвинения лидера хакеров Lurk в п...,https://lenta.ru/news/2022/09/29/oprovergli/,2022-09-29;18:35,NaN,mini
4,Пробка на мосту в Бразилии обернулась смертью ...,https://lenta.ru/news/2022/09/29/31435/,2022-09-29;18:29,NaN,mini
5,США предупреждали Украину о приближении крылат...,https://lenta.ru/news/2022/09/29/krylatye/,2022-09-29;18:28,NaN,mini
6,Пьяная российская пара забила до смерти подруг...,https://lenta.ru/news/2022/09/29/parasud/,2022-09-29;18:28,NaN,mini
7,ЦБ разрешил юрлицам покупать иностранные бумаги,https://lenta.ru/news/2022/09/29/bezogranich/,2022-09-29;18:23,NaN,mini
8,Блогерша сделала 14 татуировок на ноге за один...,https://lenta.ru/news/2022/09/29/tattoo/,2022-09-29;18:22,NaN,mini
9,Минобороны показало занятия по боевому слажива...,https://lenta.ru/news/2022/09/29/zanyatiya/,2022-09-29;18:21,NaN,mini


<!--  -->

__P.S.__

Не удалось распарсить значение атрибута `xlink:href` тега `use` — как только не пытался при помощи `xpath`... 

__КАК ЭТО СДЕЛАТЬ ПРИ ПОМОЩИ `XPATH`?__

<!--  -->